# Experiment with occupation-irrelevant gender classifiers

In [3]:
# imports
import matplotlib.pyplot as plt
import pickle as pkl
from collections import Counter
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pickle as pkl
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

bios = pkl.load(open('biosbias/data/BIOS_inferred.pkl','rb'))
data = bios
print(f'Data: {len(data)}')
# stratified split
labels_all = [bio['title'] for bio in data]
bios_data_train_val, bios_data_test = train_test_split(data, test_size=0.20, random_state=42, stratify=labels_all)

labels_train_val = [bio['title'] for bio in bios_data_train_val]
bios_data_train, bios_data_val = train_test_split(bios_data_train_val, test_size=0.25, random_state=42, stratify=labels_train_val)

with open("../../words/embeddings/OtherFormats/crawl-300d-2M.pkl", "rb") as f:
    E = pkl.load(f)
print("Loaded", len(E), "words")

Data: 397907
Loaded 1999995 words


NameError: name 'np' is not defined

In [4]:


import numpy as np
def sim(w1, w2):
    return E[w1].dot(E[w2])/np.linalg.norm(E[w1])/np.linalg.norm(E[w2])
 
sim('he', 'she')
def word_vector_featurize(text, Emb = E):
    return np.mean([Emb[w] for w in re.split(r"[\s\.\!\?\:,\"“\—\-\(\)]+", text) if len(w)>1 and w in Emb], axis=0)



In [5]:
# load chi scores
f_chi = pkl.load(open('results/f_chi.pkl','rb'))
m_chi = pkl.load(open('results/m_chi.pkl','rb'))

# threshold for determining task relevance
from scipy.stats import chi2
prob = 0.99
critical = chi2.ppf(prob, 1)
thr = critical

In [9]:
vectorizer = CountVectorizer(analyzer='word',min_df=0.001,binary=False)
X_train = vectorizer.fit_transform([p["bio"] for p in bios_data_train])
words = np.array(vectorizer.get_feature_names())


In [19]:
sorted_occs = ['accountant', 'architect', 'attorney', 'chiropractor', 'comedian', 'composer',
 'dentist', 'dietitian', 'dj', 'filmmaker', 'interior_designer', 'journalist',
 'model', 'nurse', 'painter', 'paralegal', 'pastor', 'personal_trainer',
 'photographer', 'physician', 'poet', 'professor', 'psychologist', 'rapper',
 'software_engineer', 'surgeon', 'teacher', 'yoga_teacher']
_f = Counter([p["title"] for p in bios_data_train if p["gender"]=="F"])
_m = Counter([p["title"] for p in bios_data_train if p["gender"]=="M"])
_t = Counter([p["title"] for p in bios_data_train])
fem = {t:_f[t]/float(_t[t]) for t in sorted_occs}

titles = sorted_occs.copy()
titles.sort(key=lambda t: fem[t])

G_train = [p["gender"] for p in bios_data_train]


# balanced

occs_full = np.unique([p['title'] for p in bios_data_train])
#make sample weights
sample_weights=[]
valdict = {}
for t in occs_full:
        fval = _m[t] / max(_f[t],_m[t])
        mval = _f[t] / max(_f[t],_m[t])
        valdict[t] = {}
        valdict[t]['F'] = fval
        valdict[t]['M'] = mval
    
# for :
sample_weights = [valdict[p['title']][p['gender']] for p in bios_data_train]

In [21]:
# train classifiers and save vocabularies
thr = critical
chi_gender_classers = {}
chi_vectorizers = {}
occs = ["surgeon", "software_engineer", "composer", "pastor", "comedian", "architect", "chiropractor", "accountant", "attorney", "filmmaker", "physician", "dentist", "photographer", "professor", "painter", "journalist", "poet", "personal_trainer", "teacher", "psychologist",  "model", "interior_designer", "yoga_teacher", "nurse", "dietitian"]

for occ in occs:
    print(occ)

        
    indset =[i for i,x in enumerate(f_chi[occ]) if x[0][0]<critical or m_chi[occ][i][0][0]<critical]

    print(len(list(indset)))
    irrev_chi_words = words[list(indset)]
    
    irrev_chi_vectorizer = CountVectorizer(analyzer='word',min_df=0.001,binary=False,vocabulary=irrev_chi_words)
    irrev_chi_X_train = irrev_chi_vectorizer.fit_transform([p["bio"] for p in bios_data_train])
    irrev_chi_X_val = irrev_chi_vectorizer.transform([p["bio"] for p in bios_data_val])
    irrev_chi_X_test = irrev_chi_vectorizer.transform([p["bio"] for p in bios_data_test])

    balanced_gender_clf = sklearn.linear_model.SGDClassifier(loss='log')
    balanced_gender_clf.fit(irrev_chi_X_train, G_train,sample_weight=sample_weights)
    
    chi_gender_classers[occ] = {}
    chi_gender_classers[occ][thr] = balanced_gender_clf
    chi_vectorizers[occ] = {}
    chi_vectorizers[occ][thr] = irrev_chi_vectorizer

surgeon
3297
software_engineer
3634
composer
3542
pastor
3835
comedian
3667
architect
3427
chiropractor
3779
accountant
3420
attorney
2198
filmmaker
3364
physician
1150
dentist
2236
photographer
2201
professor
1537
painter
2979
journalist
2619
poet
3166
personal_trainer
3818
teacher
2865
psychologist
2923
model
3555
interior_designer
3923
yoga_teacher
3915
nurse
3401
dietitian
4003


In [22]:
# save the classifiers
pkl.dump(chi_gender_classers,open('results/bow_chi_gender_classers_or.pkl','wb'))
pkl.dump(chi_vectorizers,open('results/bow_chi_vectorizers_or.pkl','wb'))


In [28]:
# word embedding version
import re
occupation_embeddings = {}

thr = critical
for occ in occs:
    all_words = list(chi_vectorizers[occ][thr].vocabulary_.keys())
    new_dict = {}
    for w in all_words:
        if w in E:
            new_dict[w] = E[w]
    occupation_embeddings[occ+"%.2f"%thr] = new_dict


def word_vector_featurize_occ(text, occupation, Emb = E):
    return np.mean([occupation_embeddings[occupation + "%.2f"%critical][w] for w in re.split(r"[\s\.\!\?\:,\"“\—\-\(\)]+", text) if len(w)>1 and w in occupation_embeddings[occupation+ "%.2f"%critical]], axis=0)

In [36]:

chi_we_gender_classers = {}
for occ in occs:
    print(occ)
    irrev_chi_vectorizer = chi_vectorizers[occ][thr]
    
    
    irrev_chi_X_train =  [word_vector_featurize_occ(p["bio"].lower(), occ, E) for p in bios_data_train]
    test_inds = []
    for i,p in enumerate(irrev_chi_X_train):
        if isinstance(p, np.ndarray):
            test_inds.append(i)
    irrev_chi_X_train_new = [irrev_chi_X_train[i] for i,p in enumerate(bios_data_train) if i in test_inds]
    G_train_temp = G_train[test_inds]
    balanced_gender_clf = sklearn.linear_model.SGDClassifier(loss='log')
    balanced_gender_clf.fit(irrev_chi_X_train_new, G_train_temp,sample_weight=sample_weights[test_inds])
    
    chi_we_gender_classers[occ] = {}
    chi_we_gender_classers[occ][thr] = balanced_gender_clf

surgeon


/home/adam/Dropbox/WorkProjects/venv_main/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/adam/Dropbox/WorkProjects/venv_main/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


software_engineer
composer
pastor
comedian


/home/adam/Dropbox/WorkProjects/venv_main/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/adam/Dropbox/WorkProjects/venv_main/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


architect
chiropractor
accountant
attorney


...

model


/home/adam/Dropbox/WorkProjects/venv_main/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/adam/Dropbox/WorkProjects/venv_main/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


interior_designer
yoga_teacher
nurse


/home/adam/Dropbox/WorkProjects/venv_main/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/adam/Dropbox/WorkProjects/venv_main/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


dietitian


In [37]:
# save the classifiers
pkl.dump(chi_we_gender_classers,open('results/we_chi_gender_classers_or.pkl','wb'))


In [ ]:
# validate classifiers against human scores